In [2]:
# import gc
# import torch
# gc.collect()
# torch.cuda.empty_cache()

In [1]:
import os
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import pickle
import random
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
import matplotlib.pyplot as plt
# from torchEST import EarlyStopping
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.classes_  = pickle.load(open('D:/yespeechdata/3.kt/encoder.pkl', 'rb'))

# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device = torch.device("cuda")
# device = torch.device("cpu")

ModuleNotFoundError: No module named 'mxnet'

In [4]:
#bert 모델, vocab 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /ntdev/youngri/2.INTENT/train/.cache/kobert_v1.zip
using cached model. /ntdev/youngri/2.INTENT/train/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [5]:
dataset_train = nlp.data.TSVDataset('/ntdev/youngri/2.INTENT/dataset_using/1.SKT/train_skt.tsv', field_indices=[0,1] , num_discard_samples=1)
dataset_val = nlp.data.TSVDataset('/ntdev/youngri/2.INTENT/dataset_using/1.SKT/val_skt.tsv', field_indices=[0,1], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset('/ntdev/youngri/2.INTENT/dataset_using/1.SKT/test_skt.tsv', field_indices=[0,1], num_discard_samples=1)
# print(dataset_train[0])
print('train 총 개수 : ', len(dataset_train))
# print(dataset_val[0])
print('val 총 개수 : ', len(dataset_val))
# print(dataset_test[0])
print('test 총 개수 : ', len(dataset_test))

num_epochs = 20
patience = 5
num_classes = len(np.unique([dataset_train[i][1] for i in range(len(dataset_train))]))
print('label 총 개수 : ', num_classes)
# n_trials = 6
model_dr_rate = 0.5

train 총 개수 :  25415
val 총 개수 :  7942
test 총 개수 :  6354
label 총 개수 :  279


In [6]:
# PREPOCESSING
# PREPOCESSING
# 토큰화, 패딩, 어텐션 마스크 만들어주는 BERTDataset 사용

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

class BERTDataset_sentence(Dataset):
    def __init__(self, sentence_ex, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([sentence_ex])]

    def __getitem__(self, i):
        return (self.sentences[i])

    def __len__(self):
        return (len(self.sentences))

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=num_classes,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()
    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

class EarlyStopping:
    def __init__(self, patience=3, verbose=False, delta=0, path ='checkpoint.pt'):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_acc_max = -np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_acc, model):

        score = val_acc

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_acc, model)
        elif score <= self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_acc, model)
            self.counter = 0

    def save_checkpoint(self, val_acc, model):
        if self.verbose:
            print(f'Validation acc decreased ({self.val_acc_max:.6f} --> {val_acc:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_acc_max = val_acc

In [7]:
## Setting parameters
# SKTBrain에서 제공하는 옵션 사용
max_len = 32
batch_size = 32
# warmup_ratio = 0.1
warmup_ratio = 0.1
num_epochs = 20
max_grad_norm = 0.95
learning_rate =  5e-5

def set_seed(seed=311):
    random.seed(seed);np.random.seed(seed);torch.manual_seed(seed);n_gpu = torch.cuda.device_count()
    if n_gpu > 0: torch.cuda.manual_seed_all(seed)
set_seed() 

In [8]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
#BERTDataset 클래스 이용, TensorDataset으로 만들어주기
data_train = BERTDataset(
    dataset = dataset_train # 전체 데이터
    , sent_idx = 0          # 문장 인덱스
    , label_idx = 1         # 라벨 인덱스
    , bert_tokenizer = tok  # 사용할 bert tokenizer
    , max_len = max_len     # 맥스 값
    , pad = True            # 패딩 여부
    , pair = False)         # 문장쌍 여부(ex.질문-답변)
data_test = BERTDataset(
    dataset = dataset_test
    , sent_idx = 0
    , label_idx = 1
    , bert_tokenizer = tok
    , max_len = max_len
    , pad = True
    , pair = False)

#배치 및 데이터로더 설정
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

using cached model. /ntdev/youngri/2.INTENT/train/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [11]:
for i in train_dataloader:
    print(i)

[tensor([[   2, 3484, 3892,  ...,    1,    1,    1],
        [   2, 4103, 3726,  ...,    1,    1,    1],
        [   2, 2145, 6844,  ...,    1,    1,    1],
        ...,
        [   2, 2659, 6493,  ...,    1,    1,    1],
        [   2, 4758, 2270,  ...,    1,    1,    1],
        [   2, 1280, 3484,  ...,    1,    1,    1]], dtype=torch.int32), tensor([ 9,  7, 23,  9,  9, 13, 12, 11, 13, 12,  5,  9,  9,  6,  9,  7,  9, 22,
         6,  9,  7,  6,  8,  9, 16, 11, 19, 21,  6,  9,  4,  9],
       dtype=torch.int32), tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32), tensor([180,  16,   4,  10,  22,  63,  41,  16, 240, 143,  43,  10, 248, 259,
        213, 143, 120, 262,  11,  14, 210, 240,  14,  14,  37, 187, 209,  36,
        120,   4, 129,   7], dtype=torch.int32)]
[tensor([[   2,  517, 7861,  ...,    1

In [9]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=279,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [14]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [29]:
# 모델 불러와서 쓰기
path = '/ntdev/youngri/2.INTENT/model_save/op_SKT/trial_0_op_test_03_SKT.pt'
model = BERTClassifier(bertmodel, dr_rate=0.5)
model.load_state_dict(torch.load(path))
model.eval()
device = torch.device('cpu')
model.to(device)
print('load done')

# test 데이터 준비
# bath_size를 1로 해서
new_test = nlp.data.TSVDataset('/ntdev/youngri/2.INTENT/dataset_using/1.SKT/test_skt.tsv', field_indices=[0,1], num_discard_samples=1)
infer_set = BERTDataset(new_test , 0, 1, tok, max_len, True, False)
infer_input = torch.utils.data.DataLoader(infer_set, batch_size=1, num_workers=4)

load done


In [30]:
# 테스트
val_acc = 0.0

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(infer_input)): 
  label = label.cpu().item()
  token_ids = token_ids.long().to(device) 
  segment_ids = segment_ids.long().to(device) 
  valid_length= valid_length 
  out = model(token_ids, valid_length, segment_ids)
  score = calc_accuracy(out, label)
  val_acc += calc_accuracy(out, label)
  prediction = out.cpu().detach().numpy().argmax()
  # print(label, prediction, score)

print("test acc {}".format(val_acc / (batch_id+1)))

# test acc 0.8984891406987724

/ntdev/youngri/YR_LAB/env/yr3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/6354 [00:00<?, ?it/s]

test acc 0.9195782184450739


In [20]:
test_text = '요금에 관해서 여쭤볼게요'
# test_text = str(input())

def sent_predict(test_text):
    infer_set_02 = BERTDataset_sentence(test_text, tok, max_len, True, False)
    infer_input_02 = torch.utils.data.DataLoader(infer_set_02, batch_size=1, num_workers=4)
    for token_ids, valid_length, segment_ids in infer_input_02:
        token_ids = token_ids.long().to(device) 
        segment_ids = segment_ids.long().to(device) 
        valid_length= valid_length 
        out = model(token_ids, valid_length, segment_ids)
        prediction = out.cpu().detach().numpy().argmax()
    pre_label = le.inverse_transform([prediction]).item()
    print(f'입력 문장: {test_text}\n--> 문장의 분류 예측값은 {pre_label}입니다.')

sent_predict(test_text)

입력 문장: 요금에 관해서 여쭤볼게요
--> 문장의 분류 예측값은 agent_general입니다.
